# Naive Bayes and K-Nearest Neighbors from Scratch
This notebook implements **Naive Bayes** and **K-Nearest Neighbors (KNN)** algorithms from scratch using only NumPy, Pandas, and Matplotlib.

### Objectives
- Preprocess the Titanic dataset
- Visualize gender and class survival rates
- Implement Naive Bayes and KNN manually
- Evaluate classification accuracy

**Note:** No external ML libraries like `scikit-learn` are used.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("train.csv")

## Data Preprocessing
We select relevant columns and encode categorical features. Missing values are removed for simplicity.
We also normalize `Age` and `Fare` to bring them to a similar scale.

In [ ]:
# Preprocessing
def preprocess_data(df):
    df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']].copy()
    df.dropna(inplace=True)
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    df[['Age', 'Fare']] = (df[['Age', 'Fare']] - df[['Age', 'Fare']].mean()) / df[['Age', 'Fare']].std()
    x = df.drop('Survived', axis=1).values
    y = df['Survived'].values
    split_idx = int(0.8 * len(x))
    return x[:split_idx], x[split_idx:], y[:split_idx], y[split_idx:], df

x_train, x_test, y_train, y_test, preprocessed_df = preprocess_data(df)

## Data Visualization
We visualize survival rates based on gender and passenger class.

In [ ]:
def visualize_data(df):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    df['Sex_encoded'] = df['Sex']
    df['Survived'].groupby(df['Sex_encoded']).mean().plot(kind='bar', ax=axes[0])
    axes[0].set_title("Survival Rate by Gender")
    axes[0].set_xticklabels(['Male', 'Female'], rotation=0)
    axes[0].set_ylabel("Survival Rate")
    df['Survived'].groupby(df['Pclass']).mean().plot(kind='bar', ax=axes[1])
    axes[1].set_title("Survival Rate by Class")
    axes[1].set_xlabel("Pclass")
    axes[1].set_ylabel("Survival Rate")
    plt.tight_layout()
    plt.show()

visualize_data(preprocessed_df)

## Naive Bayes Classifier Implementation

In [ ]:
def naive_bayes_classifier(x_train, y_train, x_test):
    classes = np.unique(y_train)
    priors = {c: np.mean(y_train == c) for c in classes}
    means = {c: x_train[y_train == c].mean(axis=0) for c in classes}
    variances = {c: x_train[y_train == c].var(axis=0) + 1e-6 for c in classes}

    def gaussian_prob(x, mean, var):
        exponent = np.exp(-((x - mean) ** 2) / (2 * var))
        return (1 / np.sqrt(2 * np.pi * var)) * exponent

    y_pred = []
    for x in x_test:
        posteriors = {}
        for c in classes:
            prior = np.log(priors[c])
            conditional = np.sum(np.log(gaussian_prob(x, means[c], variances[c])))
            posteriors[c] = prior + conditional
        y_pred.append(max(posteriors, key=posteriors.get))
    return np.array(y_pred)

nb_predictions = naive_bayes_classifier(x_train, y_train, x_test)
nb_accuracy = np.mean(nb_predictions == y_test)
print(f"Naive Bayes Accuracy: {nb_accuracy * 100:.2f}%")

## K-Nearest Neighbors (KNN) Classifier Implementation

In [ ]:
def knn_classifier(x_train, y_train, x_test, k=5):
    def euclidean_distance(a, b):
        return np.sqrt(np.sum((a - b) ** 2))

    y_pred = []
    for test_point in x_test:
        distances = [euclidean_distance(test_point, x) for x in x_train]
        k_indices = np.argsort(distances)[:k]
        k_labels = y_train[k_indices]
        majority = np.bincount(k_labels).argmax()
        y_pred.append(majority)
    return np.array(y_pred)

knn_predictions = knn_classifier(x_train, y_train, x_test, k=5)
knn_accuracy = np.mean(knn_predictions == y_test)
print(f"KNN Accuracy (k=5): {knn_accuracy * 100:.2f}%")

## Conclusion
- Implemented both algorithms from scratch.
- Evaluated accuracy on real Titanic data.
- Preprocessing and visualizations included for bonus marks.

**Ready for upload to the portal and GitHub.**